In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

ModuleNotFoundError: No module named 'findspark'

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

In [ ]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('homesales')


In [ ]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("SELECT year_sold, round(avg(price), 2) Avg_Price FROM homesales WHERE bedrooms == 4 GROUP BY 1 ORDER BY 1 ASC").show()


In [ ]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal placesspark.sql("SELECT date_built, round(avg(price),2) Avg_Price_3bd_3bath FROM homesales WHERE bedrooms == 3 AND bathrooms == 3 GROUP BY 1 ORDER BY 1 ASC").show()
spark.sql("SELECT date_built, round(avg(price),2) Avg_Price_3bd_3bath FROM homesales WHERE bedrooms == 3 AND bathrooms == 3 GROUP BY 1 ORDER BY 1 ASC").show()


In [ ]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("SELECT date_built, round(avg(price), 2) Avg_Big_House_Price FROM homesales WHERE bedrooms ==3 AND bathrooms==3 AND floors ==2 AND sqft_living > 2000 GROUP BY 1 ORDER BY 1 ASC").show()


In [ ]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
window = Window.orderBy(df['view'].desc())

df_result = df.groupBy("view") \
              .agg(round(avg("price"), 2).alias("Avg_Price")) \
              .withColumn("rn", row_number().over(window)) \
              .filter(col("rn") >= 350000) \
              .orderBy("view", ascending=False)

df_result.show()

print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
# 7. Cache the the temporary table home_sales.
df.createOrReplaceTempView("homesales")
df.cache()


In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

In [ ]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.
df_result = df.groupBy("view") \
              .agg(round(avg("price"), 2).alias("Avg_Price")) \
              .filter(col("Avg_Price") >= 350000) \
              .orderBy("view", ascending=False)

df_result.show()

print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy('date_built').mode('overwrite').parquet('homesales_partitioned')


In [ ]:
# 11. Read the parquet formatted data.
part_df = spark.read.format('parquet').load('homesales_partitioned')


In [ ]:
# 12. Create a temporary table for the parquet data.
part_df.createOrReplaceTempView('p_homesales')


In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()
spark.sql("""SELECT view, round(avg(price), 2) Avg_Price
FROM p_homesales
GROUP BY 1
HAVING avg(price) >= 350000
ORDER BY 1 DESC""").show()


print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table homesales")


In [ ]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('homesales')
